In [10]:
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import numpy as np
import os

np.seterr(divide='ignore', invalid='ignore')
plt.rcParams.update({'figure.max_open_warning': 0})

###############################################################################################
dir_stack = '/Users/astro/Documents/notebooks/zs/pair-m-z/stacking/'
dir_drp='/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
dir_dap = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/HYB10-MILESHC-MILESHC/'
dir_res='/Users/astro/Documents/notebooks/zs/pair-m-z/bpt/'
###############################################################################################

def plot(wave,flux,nii,ha,oiii,hb,ellcoo,i):
    
    plt.figure(figsize=(15, 15))
    ax1 = plt.subplot(221)
    ax2=plt.subplot(222)
    ax3=plt.subplot(223)
    ax4=plt.subplot(224)
    
    x=np.log10(nii/ha)
    y=np.log10(oiii/hb)
    
    ax1.plot(wave, flux, label='stacking spectral')
    ax1.set_xlabel(r"$\lambda$ (A)")
    ax1.set_ylabel(r'$10^{17}$ erg/s/cm2')
    ax1.legend()
    
    # bpt line
    x_bpt=np.arange(np.min(x)-0.01,0.2,0.02)
    y_bpt=0.61/(x_bpt-0.47)+1.19
    x_bpt1=np.arange(np.min(x)-0.01,-0.2,0.02)
    y_bpt1=0.61/(x_bpt1-0.05)+1.3
    ax2.plot(x_bpt,y_bpt,'r')
    ax2.plot(x_bpt1,y_bpt1,'k:')
        
    mask_sf=(y<(0.61/(x-0.05)+1.3))&(x<0.05)&(y<(0.61/(x-0.47)+1.19))
    x_sf=x[mask_sf]
    y_sf=y[mask_sf]
    ax2.scatter(x,y,color='dodgerblue',s=2)
    ax2.scatter(x_sf,y_sf,color='orange',s=2)
    ax2.set_xlabel('log(NII/Ha)')
    ax2.set_ylabel('log([OIII]/Hb)')
    
    # 0-1re
    mask_0_1re=(ellcoo<=1)
    print(nii.shape,mask_0_1re.shape)
    nii_1re=nii[mask_0_1re]
    ha_1re=nii[mask_0_1re]
    oiii_1re=oiii[mask_0_1re]
    hb_1re=hb[mask_0_1re]
    x_1re=np.log10(nii_1re/ha_1re)
    y_1re=np.log10(oiii_1re/hb_1re)
    mask_sf_1re=(y_1re<(0.61/(x_1re-0.05)+1.3))&(x_1re<0.05)&(y_1re<(0.61/(x_1re-0.47)+1.19))
    x_sf_1re=x_1re[mask_sf_1re]
    y_sf_1re=y_1re[mask_sf_1re]
    ax3.scatter(x_1re,y_1re,color='dodgerblue',s=2)
    ax3.scatter(x_sf_1re,y_sf_1re,color='orange',s=2,label='sf')
    ax3.plot(x_bpt,y_bpt,'r')
    ax3.plot(x_bpt1,y_bpt1,'k:')
    ax3.set_xlim(np.min(x_1re)-0.02,np.max(x_1re)+0.04)
    ax3.set_xlabel('log(NII/Ha)')
    ax3.set_ylabel('log([OIII]/Hb)')
    ax3.set_title('%s : 0-1re'%i)

    # 1-1.5re
    mask_1_15re=(ellcoo<=1.5)&(ellcoo>1)
    nii_1_15re=nii[mask_1_15re]
    ha_1_15re=nii[mask_1_15re]
    oiii_1_15re=oiii[mask_1_15re]
    hb_1_15re=hb[mask_1_15re]
    x_1_15re=np.log10(nii_1_15re/ha_1_15re)
    y_1_15re=np.log10(oiii_1_15re/hb_1_15re)
    mask_sf_1_15re=(y_1_15re<(0.61/(x_1_15re-0.05)+1.3))&(x_1_15re<0.05)&(y_1_15re<(0.61/(x_1_15re-0.47)+1.19))
    x_sf_1_15re=x_1_15re[mask_sf_1_15re]
    y_sf_1_15re=y_1_15re[mask_sf_1_15re]
    ax4.scatter(x_1_15re,y_1_15re,color='dodgerblue',s=2)
    ax4.scatter(x_sf_1_15re,y_sf_1_15re,color='orange',s=2,label='sf')
    ax4.plot(x_bpt,y_bpt,'r')
    ax4.plot(x_bpt1,y_bpt1,'k:')
    ax4.set_xlim(np.min(x_1_15re)-0.02,np.max(x_1_15re)+0.04)
    ax4.set_xlabel('log(NII/Ha)')
    ax4.set_ylabel('log([OIII]/Hb)')
    ax4.set_title('%s : 1-1.5re'%i)    
    plt.savefig(dir_res+i+'_bpt.jpg')
    plt.close()
    
    return mask_0_1re,mask_sf_1re,mask_1_15re,mask_sf_1_15re
    
def vaild(mask_region,mask_sf,i):
    
    all_num=np.sum(mask_region)
    sf_num=np.sum(mask_sf)
    return all_num,sf_num

def check_agn_remove_process():
    
    f_drp=fits.open(dir_drp)
    plateifu=f_drp[1].data.field('plateifu')

    for i in plateifu:        
        if os.path.exists(dir_stack+i+'_1re_stack.fits'):
            ###############################读数据
            f1=fits.open(dir_stack+i+'_1re_stack.fits')
            t = f1[1].data
            wave = t['wave']
            flux = t['flux']
            
            plate = i.split('-')[0]
            ifu = i.split('-')[1]

            mapf=fits.open(dir_dap+plate+'/'+ifu+'/manga-'+i+'-MAPS-HYB10-MILESHC-MILESHC.fits.gz') 
            flux_map = mapf['EMLINE_GFLUX'].data
            mask_map = mapf['EMLINE_GFLUX_MASK'].data
            ellcoo = mapf['SPX_ELLCOO'].data[1]

            mask=(mask_map!=0)
            flux_map=np.ma.array(flux_map,mask=mask)

            nii=flux_map[19]
            ha=flux_map[18]
            oiii=flux_map[13]
            hb=flux_map[11]
            ###############################
            mask_0_1re,mask_sf_1re,mask_1_15re,mask_sf_1_15re=plot(wave,flux,nii,ha,oiii,hb,ellcoo,i)
            
            
            all_01,sf_01=vaild(mask_0_1re,mask_sf_1re,i)
            all_115,sf_115=vaild(mask_1_15re,mask_sf_1_15re,i)
            

            if sf_01/all_01>0.3:
                print(i,open(dir_res+'0-1re-vaild.txt'),'a+')
            if sf_115/all_115>0.3:
                print(i,open(dir_res+'1-1.5re-vaild.txt','a+'))
            

if __name__ == '__main__':
    check_agn_remove_process()

(74, 74) (74, 74)


NameError: name 'sf_02' is not defined

In [1]:
a=2

def num(a):
    if a >1:
        return 'a'
    else:
        return 'b'

In [3]:
num(0)

'b'